In [1]:
import numpy as np
import pandas as pd
import psycopg2

In [2]:
conn = psycopg2.connect(host="localhost", database="banki_ru", user="example", password="example")

In [3]:
with conn.cursor() as cur:
    cur.execute(
        """
            SELECT
              *
            FROM banki_ru_broker
            """
    )
    res = cur.fetchall()
brokers = pd.DataFrame(res, columns=["id", "bank_name", "bank_code", "bank_id"]).set_index("id")
brokers.head()

,bank_name,bank_code,bank_id
id,,,
1,Credit Suisse,creditsuisse,10027
2,Cresco Finance,crescofinance,10207
3,General Invest,generalinvest,10221
4,GrottBjorn,grottbjorn,10124
5,HSBC Group,hsbc,10009


In [4]:
with conn.cursor() as cur:
    cur.execute(
        """
            SELECT
              *
            FROM banki_ru_insurance
            """
    )
    res = cur.fetchall()
insurance = pd.DataFrame(res, columns=["id", "bank_id", "bank_name", "bank_code"]).set_index("id")
insurance.head()

,bank_id,bank_name,bank_code
id,,,
1,10346,Абсолют Страхование,absolyut
2,10333,Авангард-Гарант,avangardgarant
3,10318,АИГ страховая компания,aigstrahovayakompaniya
4,10451,АК Барс-Мед,akbarsmed
5,10443,Ак Барс Страхование,akbarsstrahovanie


In [5]:
with conn.cursor() as cur:
    cur.execute(
        """
            SELECT
              *
            FROM banki_ru_mfo
            """
    )
    res = cur.fetchall()
mfo = pd.DataFrame(res, columns=["id", "bank_name", "bank_code", "bank_id"]).set_index("id")
mfo.head()

,bank_name,bank_code,bank_id
id,,,
1,BelkaCredit,belkacredit,10977
2,Крепкая монета,krepkaya_moneta,11410
3,Кредиттер,creditter,11298
4,Купи не копи,kupi_ne_kopi,10753
5,JoyMoney,joymoney,10802


In [6]:
df = pd.read_excel("id_other_industries.xlsx").rename({"ВК": "vk"}, axis=1)
df = df.drop_duplicates(subset=['type', 'name'])
df["old_vk"] = df["vk"]
df["vk"] = df["vk"].str.split("?", expand=True)[0].str.split("/", expand=True)[3]
df.head()

,type,id,name,URL,url_sravni,url_irec,url_otz,vk,old_vk
0,insurance,0,Абсолют Страхование,https://www.banki.ru/insurance/companies/absol...,https://www.sravni.ru/strakhovaja-kompanija/ab...,NaN,https://otzovik.com/reviews/strahovaya_kompani...,absolutins,https://vk.com/absolutins?ysclid=lak2fbpdvn279...
1,insurance,1,Авангард-Гарант,https://www.banki.ru/insurance/companies/avang...,https://www.sravni.ru/strakhovaja-kompanija/av...,NaN,https://otzovik.com/reviews/bank_avangard_russ...,NaN,NaN
2,insurance,2,АИГ страховая компания,https://www.banki.ru/insurance/companies/aigst...,https://www.sravni.ru/strakhovaja-kompanija/ai...,NaN,https://otzovik.com/reviews/strahovaya_kompani...,NaN,NaN
3,insurance,3,АК Барс-Мед,https://www.banki.ru/insurance/companies/akbar...,https://www.sravni.ru/strakhovaja-kompanija/ba...,NaN,https://otzovik.com/reviews/strahovaya_kompani...,akbars_med,https://vk.com/akbars_med?ysclid=lak2lmxlxs589...
4,insurance,4,Ак Барс Страхование,https://www.banki.ru/insurance/companies/akbar...,https://www.sravni.ru/strakhovaja-kompanija/ba...,NaN,NaN,akbarsstrah,https://vk.com/akbarsstrah?ysclid=lak2m0y54g84...


In [7]:
codes = []
for url in df["URL"]:
    if len(url.split("/")) > 2:
        codes.append(url.split("/")[-2])
    else:
        codes.append(None)
df["code"] = codes
df.head()

,type,id,name,URL,url_sravni,url_irec,url_otz,vk,old_vk,code
0,insurance,0,Абсолют Страхование,https://www.banki.ru/insurance/companies/absol...,https://www.sravni.ru/strakhovaja-kompanija/ab...,NaN,https://otzovik.com/reviews/strahovaya_kompani...,absolutins,https://vk.com/absolutins?ysclid=lak2fbpdvn279...,absolyut
1,insurance,1,Авангард-Гарант,https://www.banki.ru/insurance/companies/avang...,https://www.sravni.ru/strakhovaja-kompanija/av...,NaN,https://otzovik.com/reviews/bank_avangard_russ...,NaN,NaN,avangardgarant
2,insurance,2,АИГ страховая компания,https://www.banki.ru/insurance/companies/aigst...,https://www.sravni.ru/strakhovaja-kompanija/ai...,NaN,https://otzovik.com/reviews/strahovaya_kompani...,NaN,NaN,aigstrahovayakompaniya
3,insurance,3,АК Барс-Мед,https://www.banki.ru/insurance/companies/akbar...,https://www.sravni.ru/strakhovaja-kompanija/ba...,NaN,https://otzovik.com/reviews/strahovaya_kompani...,akbars_med,https://vk.com/akbars_med?ysclid=lak2lmxlxs589...,akbarsmed
4,insurance,4,Ак Барс Страхование,https://www.banki.ru/insurance/companies/akbar...,https://www.sravni.ru/strakhovaja-kompanija/ba...,NaN,NaN,akbarsstrah,https://vk.com/akbarsstrah?ysclid=lak2m0y54g84...,akbarsstrahovanie


In [8]:
isuarance_ids = []
insurance_df = df[df["type"] == "insurance"]
for i, row in insurance_df.iterrows():
    insuarance_code = row["code"]
    if insuarance_code in insurance["bank_code"].values:
        isuarance_ids.append(insurance[insurance["bank_code"] == insuarance_code].iloc[0, 0])
    else:
        isuarance_ids.append(None)
        print(insuarance_code, row["name"])

astrametall Астра-Металл
bksstrahovaniezhizni БКС Страхование жизни
gic Джиайси перестрахование
intouchstrahovanie Интач Страхование
mango Манго Иншуринг
v-vpovs НКО «ВОВС»
otvet-ins НКО ПОВС «Ответственность»
nkoparus НКО ПОВС «Парус»
rt-ovs НКО ПОВС «РТ - ВС»
micnc НКО ПОВС «Северная столица»
povsto НКО «ПОВСТО»
uyutnyjdom-1 НКО ПОВС «Уютный Дом-1»
rncb РНКБ Страхование
pulse СК «Пульс»
sosetezheneralstrahovaniezhizni Сосьете Женераль Страхование Жизни


In [9]:
brokers_ids = []
brokers_df = df[df["type"] == "broker_i_uk"]
for i, row in brokers_df.iterrows():
    brokers_code = row["code"]
    if brokers_code in brokers["bank_code"].values:
        brokers_ids.append(brokers[brokers["bank_code"] == brokers_code].iloc[0, 2])
    else:
        brokers_ids.append(None)
        print(brokers_code, row["name"])

sberfn Современные Фонды Недвижимости
sberbank-am Первая
vtbcapital-am Вим Инвестиции
vtbcapital-pr ВТБ Капитал Пенсионный резерв
region-rd Регион Девелопмент
region-am Регион Эссет Менеджмент
trust-region Регион Траст
ingosstrah-invest Ингосстрах-Инвестиции
gpb-am ААА Управление Капиталом
capital-am Мой Капитал
alfacapital Альфа-Капитал
open-am Открытие
leader Лидер
tkbip ТКБ Инвестмент Партнерс
progressinvest Прогрессивные инвестиционные идеи
tfg ТФГ
sistema-capital Система Капитал
raiffeisencapital Райффайзен Капитал
rshb-am РСХБ Управление Активами
llcams Эссет Менеджмент Солюшнс
finex-etf FinEx
aclass А класс капитал
agana Агана
uralsib-am БКС Управление благосостоянием
bcs БКС
svinin Свиньин и Партнеры
veles-capitalam УК Велес Менеджмент
ew-mc Восток-Запад
akbarscapital Ак Барс Капитал
sputnik-am Спутник-Управление Капиталом
veles-trust Велес Траст
tinkoffcapital Тинькофф Капитал
promsvyaz УК Промсвязь
rwminvest RWM Capital
aton-m Атон-менеджмент
kspcapital-am КСП Капитал
rfc-cap

In [10]:
mfo_ids = []
mfo_df = df[df["type"] == "microfin"]
for i, row in mfo_df.iterrows():
    mfo_code = row["code"]
    if mfo_code in mfo["bank_code"].values:
        mfo_ids.append(mfo[mfo["bank_code"] == mfo_code].iloc[0, 2])
    else:
        mfo_ids.append(None)
        print(mfo_code, row["name"])

revotekhnologii Мокка
vivadengi VIVA Деньги
centrofinans Центрофинанс
bystrozaym БЫСТРОЗАЙМ
vdplatinum Выручай-Деньги
credit_smile CreditSmile
dengivdolg Деньги в долг
daem_zaem Финансовая уверенность
microdengi Микроденьги


In [11]:
res = []
res.extend(isuarance_ids)
res.extend(brokers_ids)
res.extend(mfo_ids)
df["bank_id"] = res

In [12]:
df.to_csv("id_other_industries.csv", index=False)

In [92]:
for url in ["url_sravni", "url_irec", "vk"]:
    tmp = df[["bank_id", "name", url]]
    
    tmp = tmp[tmp[url].notna() & tmp["bank_id"].notna()]
    if url == "vk":
        tmp = tmp[~tmp[url].isin(["freedom_finance"])]
    tmp = tmp.to_numpy()
    with open(f"{url}_other_industries.npy", "wb") as f:
        np.save(f, tmp)